# Helper functions

In [19]:
# Load the test dataset
import json
import os

def load_json_from_folder(folder_path):
  """
  Loads all JSON files from a specified folder.

  Args:
    folder_path: The path to the folder containing the JSON files.

  Returns:
    A list containing all JSON objects from the files.
  """

  all_data = []
  for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
      filepath = os.path.join(folder_path, filename)
      try:
        with open(filepath, 'r') as f:
          data = json.load(f)
          all_data.extend(data)
      except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {filename}: {e}")
  return all_data

In [20]:
import ast

def validate_code(code_str):
  """
  Validates Python code and returns error information if any.

  Args:
    code_str: The Python code as a string.

  Returns:
    None if the code is valid, otherwise a string describing the syntax error.
  """
  try:
    ast.parse(code_str)
    return None  # No error
  except SyntaxError as e:
    return str(e)  # Return the error message as a string

# Evaluate GitHub Python

## Load the dataset

In [21]:
github_python_dataset = "../github-python-test"
all_data = load_json_from_folder(github_python_dataset)

In [22]:
total_records = len(all_data)
first_code = all_data[0]['src']['string_format']

print(f"Total number of objects: {total_records}")
print(f"First code:\n {first_code}")
print(f"Error (if any): {validate_code(first_code)}")

Total number of objects: 37639
First code:
 def test_delitem_keyerror ( self ) :
    e = EntryBase ( req_ ( )
    del e [ "str" ]

Error (if any): '(' was never closed (<unknown>, line 2)


## Prepare the model

In [2]:
import torch
import sys
import os
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, PeftModel, get_peft_model
from transformers import TrainingArguments, Trainer

In [38]:
# Set HF token as env variable


In [3]:
# Prepare token
torch.cuda.empty_cache()
hf_token = os.environ.get('HF_TOKEN')

In [25]:
# Prepare instruction
python_syntax_fixer_instruction = "You are an expert Python code fixer. \
             You will receive input in the following format: \n\n \
             [Fix] | <error code>\n \
             <python code snippet>\n\n \
             Your task is to ONLY provide the corrected Python code with NO explanations or additional text. \n \
             Do not include the original error code in your response and do not format the code. \
             Treat the code snippet as regular text."

In [28]:
# Function for fixing code
def fix_code(code_snippet, error_code):
    messages = [
        {"role": "system", "content": python_syntax_fixer_instruction}
    ]
    messages.append({"role": "user", "content": f"[Fix] | {error_code}\n{code_snippet}"})
    outputs = pipe(messages, max_new_tokens=256)
    return outputs[0]["generated_text"][-1]["content"]

In [17]:
# Load the model and instruction
instruct_model_id = "meta-llama/Llama-3.2-3B-Instruct"

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    token=hf_token,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
fixed_code = fix_code(first_code, validate_code(first_code))
fixed_code

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'def test_delitem_keyerror ( self ) :\n    e = EntryBase ( req_ () )\n    del e[ "str" ]'

In [36]:
validate_code(fixed_code)

In [37]:
first_code

'def test_delitem_keyerror ( self ) :\n    e = EntryBase ( req_ ( )\n    del e [ "str" ]\n'

# WIP

In [ ]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
messages = [
    {"role": "system", "content": "You are a Python syntax error correction expert. You will receive code snippets with syntax errors, prefixed with [Fix]. Each input will have two parts separated by a vertical bar (|). You will only fix the provided code, without any additional explanation."},
]
outputs = pipe(
    messages,
    max_new_tokens=128,
)
print(outputs[0]["generated_text"][-1])

In [11]:
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

error_code = "SyntaxError: invalid syntax"
code_snippet = """
def greet(name):
  print("Hello, " + name 

greet("Alice")
"""

messages = [
    { 
        "role": "system", 
        "content": \
            "You are an expert Python code fixer. \
             You will receive input in the following format: \n\n \
             [Fix] | <error code>\n \
             <python code snippet>\n\n \
             Your task is to ONLY provide the corrected Python code with NO explanations or additional text. \n \
             Do not include the original error code in your response and do not format the code. \
             Treat the code snippet as regular text."
    }
]

messages.append({"role": "user", "content": f"""
[Fix] | {error_code}
{code_snippet}
"""})

outputs = pipe(messages, max_new_tokens=256)
print(outputs[0]["generated_text"][-1])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'role': 'assistant', 'content': 'def greet(name):\n  print("Hello, " + name)'}


In [13]:
fixed_code = outputs[0]["generated_text"][-1]['content']
fixed_code

'def greet(name):\n  print("Hello, " + name)'

In [14]:
validate_code(code_snippet)

"'(' was never closed (<unknown>, line 3)"

In [15]:
validate_code(fixed_code)